## Simple GenAI app with langchain:

- load the data
- get docs
- divide our text docs in the smaller chunks
- create vectors
- vector embeddings
- store embeddings in vectorstoreDB
- infernece the vector storeDB using LLM based on similarity seach
- deploy the Gen AI app in cloud

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_TRACING_V2"]= "True"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

: 

In [9]:
%pip install -qU langchain_community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


error: uninstall-no-record-file

× Cannot uninstall langchain-community 0.3.15
╰─> The package's contents are unknown: no RECORD file was found for langchain-community.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps langchain-community==0.3.15


In [10]:
## data ingestion from the website and scrape the data:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")


In [11]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memor

In [13]:
# split the docs in to text chunks:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents=text_splitter.split_documents(docs)
documents


[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a 

In [14]:
# embed the text chunks:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()


In [ ]:
%pip install faiss-cpu

   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ------ --------------------------------- 2.4/13.8 MB 11.2 MB/s eta 0:00:02
   -------------- ------------------------- 5.0/13.8 MB 11.6 MB/s eta 0:00:01
   --------------------- ------------------ 7.3/13.8 MB 11.3 MB/s eta 0:00:01
   ---------------------------- ----------- 9.7/13.8 MB 11.4 MB/s eta 0:00:01
   ---------------------------------- ----- 12.1/13.8 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------  13.6/13.8 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------  13.6/13.8 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------  13.6/13.8 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------- 13.8/13.8 MB 7.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
# store the embeddings in the database: FAISS DB:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)


In [19]:
vectorstoredb

In [23]:
# check by retrieving the similar documents: from the vectorstoreDB
## query from vectorstoredb:
query = "Architecture"
result = vectorstoredb.similarity_search(query)
print(result[0].page_content)

Contributing​
Check out the developer's guide for guidelines on contributing and help getting your dev environment set up.Edit this pageWas this page helpful?NextTutorialsArchitectureGuidesTutorialsHow-to guidesConceptual guideIntegrationsAPI referenceEcosystem🦜🛠️ LangSmith🦜🕸️ LangGraphAdditional resourcesVersionsSecurityContributingCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2025 LangChain, Inc.


In [24]:
# initialize llm:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-4o')

In [27]:
# Retrieval chain: documents chain:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001FFFB34C7C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001FFFB945090>, root_client=<openai.OpenAI object at 0x000001FFFB8A5FC0>, root_async_client=<openai.AsyncOpenAI object at 0x000001FFFB34C7F0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_do

In [29]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

'LangSmith has two usage limits: total traces and extended traces. These limits correspond to the two metrics that are tracked on their usage graph.'

In [30]:
vectorstoredb

In [36]:
# Retriever:
## input -> query -> retrieve -> output
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FFF9C04730>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [37]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'Why is LangChain beneficial for developing applications?\n\nLangChain provides a robust framework for building applications by utilizing its open-source components and third-party integrations. It offers standard interfaces for large language models, embeddings, and vector stores with support for a wide range of providers. LangGraph enables the creation of stateful agents with streaming and human-in-the-loop capabilities. LangSmith facilitates the inspection, monitoring, and optimization of applications for seamless productionization and deployment into production-ready APIs and Assistants via the LangGraph Platform. These features make LangChain a comprehensive solution for developing advanced AI applications.'

In [38]:
response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='9d99b0ed-6404-457f-84e7-23b9f5f2fd83', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='storesWhy LangChain?Ecosystem🦜🛠️ LangSmith🦜🕸️ LangGraphVersionsv0.3v0.2Pydantic compatibilityMigrating from v0.0 chainsHow to migrate from v0.0 chainsMigrating from ConstitutionalChainMigrating from ConversationalChainMigrating from ConversationalRetrievalChainMigrating from LLMChainMigrating from LLMMathChainMigrating from LLMRouterChainMigrating from MapReduceDocumentsChainMigrating from MapRerankDocumentsChainMigrating from MultiPromptChainMigrating from RefineDocumentsChainMigrating from RetrievalQAMigrating from StuffDocumentsChainUpgrading to LangGraph memoryHow to migrate to LangGraph memoryHo

In [39]:
response['context']

[Document(id='9d99b0ed-6404-457f-84e7-23b9f5f2fd83', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='storesWhy LangChain?Ecosystem🦜🛠️ LangSmith🦜🕸️ LangGraphVersionsv0.3v0.2Pydantic compatibilityMigrating from v0.0 chainsHow to migrate from v0.0 chainsMigrating from ConstitutionalChainMigrating from ConversationalChainMigrating from ConversationalRetrievalChainMigrating from LLMChainMigrating from LLMMathChainMigrating from LLMRouterChainMigrating from MapReduceDocumentsChainMigrating from MapRerankDocumentsChainMigrating from MultiPromptChainMigrating from RefineDocumentsChainMigrating from RetrievalQAMigrating from StuffDocumentsChainUpgrading to LangGraph memoryHow to migrate to LangGraph memoryHow to use BaseChatMessageHistory with LangGraphMigrating off ConversationBufferMemor